In [1]:
# metrics

# same as the build-in metric, I believe
def cat_acc(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))


def macroPrec(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    positives = K.sum(K.round(y_pred), axis=0)
    precision = true_positives / (positives + K.epsilon())

    macroPrec = K.mean( precision )
    
    return macroPrec


def macroF1(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    
    actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    precision = true_positives / (positives + K.epsilon())
    
    macroF1 = 2 * K.mean( precision*recall / (precision + recall + K.epsilon()) )
    
    return macroF1


def macroRecall(y_true, y_pred):
    
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    actual_ones = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    macroRecall = K.mean( recall )
    
    return macroRecall

In [ ]:
# loss functions

# gives results very close but not exactly equal to the buil-in categorical_crossentropy
# (that's not intentional, I'm not sure what's the difference)
def cat_cross(y_true, y_pred):
    return -K.mean(K.log( K.sum(y_true * y_pred, axis=-1) + K.epsilon() ))


def fuzzy_macroF1_flip(y_true, y_pred):

    true_positives = K.sum(y_true * y_pred, axis=0)
    
    actual_ones = K.sum(y_true, axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    positives = K.sum(y_pred, axis=0)
    precision = true_positives / (positives + K.epsilon())
    
    macroF1 = 2 * K.mean( precision*recall / (precision + recall + K.epsilon()) ) 
    
    return 1 - macroF1

def my_cross(y_true, y_pred):
    
    class_entropy = -K.mean(y_true * K.log(y_pred+K.epsilon()), axis=0)
    weighted_entropy = K.mean(class_weights * class_entropy)
    return weighted_entropy


def my_loss(y_true, y_pred):
    # weighted-avg(fuzzy_macroF1_flip, my_cross) with arbitrary weights
    
    true_positives = K.sum(y_true * y_pred, axis=0)
    
    actual_ones = K.sum(y_true, axis=0)
    recall = true_positives / (actual_ones + K.epsilon())

    positives = K.sum(y_pred, axis=0)
    precision = true_positives / (positives + K.epsilon())
    
    macroF1 = 2 * K.mean( precision*recall / (precision + recall + K.epsilon()) )
    
    class_entropy = -K.mean(y_true * K.log(y_pred+K.epsilon()), axis=0)
    weighted_entropy = K.mean(class_weights * class_entropy)
    
    return 0.9 * (1 - macroF1) + 0.1 * weighted_entropy

In [ ]:
CUSTOM_OBJECTS = {'cat_acc': cat_acc,
                  'macroPrec': macroPrec,
                  'macroF1': macroF1,
                  'macroRecall': macroRecall,
                  'cat_cross': cat_cross,
                  'fuzzy_macroF1_flip': fuzzy_macroF1_flip,
                  'my_cross': my_cross,
                  'my_loss': my_loss
                 }